In [ ]:
import os
import sys 
sys.path.append("..")
from i2e import *
from tqdm import tqdm
from torchvision import datasets
from torchvision.transforms import v2

In [ ]:
ori_dataset_path = '/ssd/Datasets/ImageNet/'
i2e_dataset_path = '/ssd/Datasets/I2E-ImageNet/'
batch_size = 128
workers = 28

In [ ]:
net = I2E(ratio=0.12)
net.eval()

In [ ]:
class I2E_ImageNet(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None, root_out=i2e_dataset_path, train=True):
        super(I2E_ImageNet, self).__init__(root, transform, target_transform)
        self.root_out = os.path.join(root_out, 'train' if train else 'val')

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        img_event = net(sample.unsqueeze(0)).squeeze()
        path_split = path.split('/')
        save_path = os.path.join(self.root_out, path_split[-2])
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)
        np.savez_compressed(os.path.join(save_path, f'''{path_split[-1].split('.')[0]}'''), img_event.numpy().astype(bool))

        return img_event, target

In [ ]:
transform_train = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((224, 224)),
    v2.ToDtype(torch.float32, scale=True),
])

transform_test = v2.Compose([
    v2.PILToTensor(),
    v2.Resize(256),
    v2.CenterCrop(224),
    v2.ToDtype(torch.float32, scale=True),
])

train_dataset = I2E_ImageNet(root=ori_dataset_path + 'train', transform=transform_train, train=True)
val_dataset = I2E_ImageNet(root=ori_dataset_path + 'val', transform=transform_test, train=False)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

In [ ]:
for i, (images, target) in enumerate(tqdm(train_loader)):
    pass

In [ ]:
for i, (images, target) in enumerate(tqdm(val_loader)):
    pass